In [13]:
pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
pip install pandas pyarrow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import requests
import pandas as pd
import json
from datetime import datetime, timedelta

#### CMD para definir as datas

In [17]:
today = datetime.today()

if today.weekday() == 5:   
    today -= timedelta(days = 1)
    
elif today.weekday() == 6:    
    today -= timedelta(days = 2)

ten_days_before = today
i = 0

while i != 9:
    if ten_days_before.weekday() < 5:
        i += 1

    ten_days_before -= timedelta(days = 1)

#### Conexão com a API e Absorção dos Dados (.json)

In [18]:
API_KEY = "19HYD6hzI86Hqfx8sTvN4fzh0wrBb4oyPlORREtn"

ENDPOINT = "https://api.stockdata.org/v1/data/intraday"

params = {
    "symbols": "AMZN,TSLA",
    "interval": "hour",
    "api_token": API_KEY,
    "date_from": ten_days_before.strftime('%Y-%m-%d'),
    "date_to": today.strftime('%Y-%m-%d')
}

response = requests.get(ENDPOINT, params=params)

if response.status_code == 200:
    
    data = response.json()

    caminho = "bronze/stock_data_bronze.json"

    with open(caminho, "w", encoding="utf-8") as arquivo:
        json.dump(data, arquivo, indent=4) 

else:
    print(f"Erro: {response.status_code}, {response.text}")

#### Limpeza e transformação dos dados (.parquet)

In [19]:
with open("bronze/stock_data_bronze.json", "r") as file:
    data = json.load(file)

df = pd.json_normalize(data["data"])

df.to_parquet("silver/stock_data_silver.parquet", engine="pyarrow", index=False)

#### Análise dos Dados

In [20]:
df = pd.read_parquet("silver/stock_data_silver.parquet", engine="pyarrow")

df[["dia", "hora"]] = df['date'].str.split('T',expand=True)

df = df.groupby(["ticker", "dia"]).agg(preco_max=("data.close", "first"), preco_min=("data.open", "last")).reset_index()

df["preco"] = (df["preco_max"] + df["preco_min"])/2

df = df.rename(columns={"ticker": "codigo_acao"})[["codigo_acao", "dia", "preco"]]

df['dia'] = pd.to_datetime(df['dia'])

In [21]:
df.to_parquet("gold/stock_data_gold.parquet", engine="pyarrow", index=False)

In [22]:
display(df)

,codigo_acao,dia,preco
0,AMZN,2024-12-03,211.885
1,AMZN,2024-12-04,217.025
2,AMZN,2024-12-05,219.375
3,AMZN,2024-12-06,223.910
4,AMZN,2024-12-09,226.680
5,AMZN,2024-12-10,225.695
6,AMZN,2024-12-11,228.330
7,AMZN,2024-12-12,229.280
8,AMZN,2024-12-13,227.955
